## Sistema de Recomendación

In [103]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import math
import re


# Visualizaciones
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Para modelos NLP
# -----------------------------------------------------------------------
import spacy
from nltk.corpus import stopwords
import nltk

from nltk.sentiment import SentimentIntensityAnalyzer

import contractions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.decomposition import PCA
# Configuración de pandas
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)
# Otros
# -----------------------------------------------------------------------
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")

In [104]:
artist_details = pd.read_csv("datos/detalles_artistas.csv")
movie_details = pd.read_csv("datos/detalles_peliculas.csv")
artist_movie = pd.read_csv("datos/pelicula_artista.csv")
movies = pd.read_csv("datos/peliculas.csv")

In [105]:
movies_complete = movies.merge(movie_details, right_on="id_pelicula", left_on="Id")
movies_complete.reset_index(inplace=True, drop=True)

In [106]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

In [107]:
def get_embedding(texts):
    """
    Genera embeddings para una lista de textos utilizando un modelo preentrenado.

    Esta función utiliza un tokenizer y un modelo de lenguaje preentrenados (como los de 
    Hugging Face) para convertir textos en representaciones numéricas (embeddings). 
    Los embeddings se calculan tomando la media de las activaciones del último estado 
    oculto del modelo.

    Parámetros:
    texts (list[str] o str): Lista de textos (o un único texto) para los cuales se 
                             generarán los embeddings.

    Devuelve:
    torch.Tensor: Tensor que contiene los embeddings generados. La forma del tensor será 
                  (N, D), donde N es el número de textos y D es la dimensión del embedding 
                  generado por el modelo.

    Ejemplo:
    >>> texts = ["Hola, ¿cómo estás?", "Esto es un ejemplo."]
    >>> embeddings = get_embedding(texts)
    >>> print(embeddings.shape)
    torch.Size([2, 768])  # Suponiendo que el modelo devuelve embeddings de 768 dimensiones.
    """
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(axis=1)
    return embeddings



In [108]:
descriptions = movies_complete[["Id", "argum_imbd"]]
descriptions.set_index("Id", inplace=True)
# descriptions = descriptions.unstack()

In [109]:
descriptions_no_unknown = descriptions[descriptions["argum_imbd"] != "Desconocido"]

In [110]:
embedded_desc = get_embedding(list(descriptions_no_unknown["argum_imbd"]))

In [111]:
df_embedded = pd.DataFrame(embedded_desc, columns = [f"emb_{i}" for i in range(1, embedded_desc.shape[1]+1)])

In [112]:
df_embedded.insert(column="id", loc = 0, value = descriptions_no_unknown["argum_imbd"].index)

In [113]:
df_embedded.set_index("id", inplace = True)
df_embedded.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,emb_11,emb_12,emb_13,emb_14,emb_15,emb_16,emb_17,emb_18,emb_19,emb_20,emb_21,emb_22,emb_23,emb_24,emb_25,emb_26,emb_27,emb_28,emb_29,emb_30,emb_31,emb_32,emb_33,emb_34,emb_35,emb_36,emb_37,emb_38,emb_39,emb_40,emb_41,emb_42,emb_43,emb_44,emb_45,emb_46,emb_47,emb_48,emb_49,emb_50,emb_51,emb_52,emb_53,emb_54,emb_55,emb_56,emb_57,emb_58,emb_59,emb_60,emb_61,emb_62,emb_63,emb_64,emb_65,emb_66,emb_67,emb_68,emb_69,emb_70,emb_71,emb_72,emb_73,emb_74,emb_75,emb_76,emb_77,emb_78,emb_79,emb_80,emb_81,emb_82,emb_83,emb_84,emb_85,emb_86,emb_87,emb_88,emb_89,emb_90,emb_91,emb_92,emb_93,emb_94,emb_95,emb_96,emb_97,emb_98,emb_99,emb_100,emb_101,emb_102,emb_103,emb_104,emb_105,emb_106,emb_107,emb_108,emb_109,emb_110,emb_111,emb_112,emb_113,emb_114,emb_115,emb_116,emb_117,emb_118,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127,emb_128,emb_129,emb_130,emb_131,emb_132,emb_133,emb_134,emb_135,emb_136,emb_137,emb_138,emb_139,emb_140,emb_141,emb_142,emb_143,emb_144,emb_145,emb_146,emb_147,emb_148,emb_149,emb_150,emb_151,emb_152,emb_153,emb_154,emb_155,emb_156,emb_157,emb_158,emb_159,emb_160,emb_161,emb_162,emb_163,emb_164,emb_165,emb_166,emb_167,emb_168,emb_169,emb_170,emb_171,emb_172,emb_173,emb_174,emb_175,emb_176,emb_177,emb_178,emb_179,emb_180,emb_181,emb_182,emb_183,emb_184,emb_185,emb_186,emb_187,emb_188,emb_189,emb_190,emb_191,emb_192,emb_193,emb_194,emb_195,emb_196,emb_197,emb_198,emb_199,emb_200,emb_201,emb_202,emb_203,emb_204,emb_205,emb_206,emb_207,emb_208,emb_209,emb_210,emb_211,emb_212,emb_213,emb_214,emb_215,emb_216,emb_217,emb_218,emb_219,emb_220,emb_221,emb_222,emb_223,emb_224,emb_225,emb_226,emb_227,emb_228,emb_229,emb_230,emb_231,emb_232,emb_233,emb_234,emb_235,emb_236,emb_237,emb_238,emb_239,emb_240,emb_241,emb_242,emb_243,emb_244,emb_245,emb_246,emb_247,emb_248,emb_249,emb_250,emb_251,emb_252,emb_253,emb_254,emb_255,emb_256,emb_257,emb_258,emb_259,emb_260,emb_261,emb_262,emb_263,emb_264,emb_265,emb_266,emb_267,emb_268,emb_269,emb_270,emb_271,emb_272,emb_273,emb_274,emb_275,emb_276,emb_277,emb_278,emb_279,emb_280,emb_281,emb_282,emb_283,emb_284,emb_285,emb_286,emb_287,emb_288,emb_289,emb_290,emb_291,emb_292,emb_293,emb_294,emb_295,emb_296,emb_297,emb_298,emb_299,emb_300,emb_301,emb_302,emb_303,emb_304,emb_305,emb_306,emb_307,emb_308,emb_309,emb_310,emb_311,emb_312,emb_313,emb_314,emb_315,emb_316,emb_317,emb_318,emb_319,emb_320,emb_321,emb_322,emb_323,emb_324,emb_325,emb_326,emb_327,emb_328,emb_329,emb_330,emb_331,emb_332,emb_333,emb_334,emb_335,emb_336,emb_337,emb_338,emb_339,emb_340,emb_341,emb_342,emb_343,emb_344,emb_345,emb_346,emb_347,emb_348,emb_349,emb_350,emb_351,emb_352,emb_353,emb_354,emb_355,emb_356,emb_357,emb_358,emb_359,emb_360,emb_361,emb_362,emb_363,emb_364,emb_365,emb_366,emb_367,emb_368,emb_369,emb_370,emb_371,emb_372,emb_373,emb_374,emb_375,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,emb_384,emb_385,emb_386,emb_387,emb_388,emb_389,emb_390,emb_391,emb_392,emb_393,emb_394,emb_395,emb_396,emb_397,emb_398,emb_399,emb_400,emb_401,emb_402,emb_403,emb_404,emb_405,emb_406,emb_407,emb_408,emb_409,emb_410,emb_411,emb_412,emb_413,emb_414,emb_415,emb_416,emb_417,emb_418,emb_419,emb_420,emb_421,emb_422,emb_423,emb_424,emb_425,emb_426,emb_427,emb_428,emb_429,emb_430,emb_431,emb_432,emb_433,emb_434,emb_435,emb_436,emb_437,emb_438,emb_439,emb_440,emb_441,emb_442,emb_443,emb_444,emb_445,emb_446,emb_447,emb_448,emb_449,emb_450,emb_451,emb_452,emb_453,emb_454,emb_455,emb_456,emb_457,emb_458,emb_459,emb_460,emb_461,emb_462,emb_463,emb_464,emb_465,emb_466,emb_467,emb_468,emb_469,emb_470,emb_471,emb_472,emb_473,emb_474,emb_475,emb_476,emb_477,emb_478,emb_479,emb_480,emb_481,emb_482,emb_483,emb_484,emb_485,emb_486,emb_487,emb_488,emb_489,emb_490,emb_491,emb_492,emb_493,emb_494,emb_495,emb_496,emb_497,emb_498,emb_499,emb_500,emb_501,emb_502,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511,emb_512,emb_513,emb

In [114]:
df_similarity = pd.DataFrame(cosine_similarity(df_embedded), columns = df_embedded.index, index = df_embedded.index)

In [115]:
movie_dict = movie_details[["titulo", "id_pelicula"]].set_index("id_pelicula").to_dict()["titulo"]

In [120]:
ratings = movie_details.set_index("id_pelicula")["cali_imdb"]

genres = movies_complete.set_index("id_pelicula")["generos"]

years = movies_complete.set_index("id_pelicula")["Año"]

In [150]:
def get_recs(id, n_recs=10, print_prod=True):
    if print_prod:
        print(movie_dict[id])
    try:
        
        similarities = df_similarity[id].rename("similarity").sort_values(ascending=False).iloc[1:].reset_index()
        similarities["title"] = similarities["id"].map(movie_dict)
        similarities = similarities.merge(right=ratings, left_on="id", right_on="id_pelicula")
        print("Recomendación basada en la descripción de la película")
        return similarities.head(n_recs)
    except:
        genre = genres[id]
        year = years[id]
        print(f"No hay descripción, recomendación basada en género : {genre} y años alrededor de {year}")
        cond = (movies_complete["Año"].between(left = year-2, right=year+2)) & (movies_complete["generos"] == genre)
        return movies_complete[cond].sort_values(by = "cali_imdb", ascending = False).set_index("id_pelicula")[["titulo","cali_imdb"]].head(n_recs)

In [151]:
get_recs("tt0334294")

La sangre de un valiente
No hay descripción, recomendación basada en género : Action y años alrededor de 1993


,titulo,cali_imdb
id_pelicula,,
tt0334294,La sangre de un valiente,6.7
tt0280889,Maanthrikam,6.3
tt0330855,Santo: la leyenda del enmascarado de plata,6.3
tt0349971,Pitong Gamol,5.9
tt0290183,Elaan,5.1
tt0344301,Sarphira,5.0
tt0290213,Hum Hain Bemisaal,4.4
tt0334765,Young-guwa gongryong Zzu-Zzu,4.2
tt0277621,Cesta peklem,3.7


In [154]:
get_recs("tt0330019")

Azaad Desh Ke Gulam
Recomendación basada en la descripción de la película


,id,similarity,title,cali_imdb
0,tt0297241,0.907716,Kyo Kii... Main Jhuth Nahin Bolta,5.4
1,tt0286936,0.907063,Salaakhen,5.4
2,tt0290098,0.903553,Varnapakittu,5.8
3,tt0454396,0.900704,Aryan: Unbreakable,4.4
4,tt0339252,0.897286,Khadgam,7.8
5,tt3527068,0.896299,A Rainy Day,5.0
6,tt0286772,0.890893,Kudrat,4.0
7,tt0454429,0.889769,Chingaari,4.7
8,tt14142640,0.888598,Bannerghatta,5.2
9,tt0290183,0.885810,Elaan,5.1
